# Lesson 2 : Building a Chat Engine with Conversation History

# Building a Chat Engine with Conversation History

Welcome to the second unit of our course on building a RAG-powered chatbot! In the previous lesson, we built a document processor that forms the retrieval component of our RAG system. Today, we'll focus on the conversational aspect by creating a chat engine that can maintain conversation history.

While our document processor is excellent at finding relevant information, a complete RAG system needs a way to interact with users in a natural, conversational manner. This is where our chat engine comes in. The chat engine is responsible for managing the conversation flow, formatting prompts with relevant context, and maintaining a history of the interaction.

## Understanding the Chat Engine

The chat engine we'll build today will:

* Manage interactions with the language model
* Optionally maintain a history of the conversation for display or logging
* Format prompts with relevant context from our document processor
* Provide methods to reset the conversation history when needed

By the end of this lesson, you'll have a fully functional chat engine that can be integrated with the document processor we built previously to create a complete RAG system.

---

## Creating the ChatEngine Class Structure

Let's begin by setting up the basic structure of our ChatEngine class. This class will encapsulate all the functionality needed for managing conversations with the language model.

```python
from langchain_openai import ChatOpenAI
from langchain.schema.messages import SystemMessage, HumanMessage, AIMessage
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

class ChatEngine:
    def __init__(self):
        self.chat_model = ChatOpenAI()
        self.system_message = (
            "You are a helpful assistant that ONLY answers questions based on the "
            "provided context. If no relevant context is provided, do NOT answer the "
            "question and politely inform the user that you don't have the necessary "
            "information to answer their question accurately."
        )
        
        # Define the prompt template with explicit system and human messages
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(self.system_message),
            HumanMessagePromptTemplate.from_template(
                "Context:\n{context}\n\nQuestion: {question}"
            )
        ])
        
        # Optionally, keep conversation history for display/logging only
        self.conversation_history = []
```

### Key points in this initialization:

* **Chat Model**: We initialize `self.chat_model` using `ChatOpenAI()` to create an instance of the OpenAI chat model for generating responses.
* **System Message**: We define strict instructions that guide the AI's behavior, telling it to only answer questions based on provided context and to politely decline if no relevant context is available.
* **Prompt Template**: We use `ChatPromptTemplate.from_messages()` to explicitly define both the system and human message templates. The system message sets the assistant's behavior, and the human message template includes placeholders for context and question.
* **Conversation History**: We initialize an empty list to optionally keep track of the conversation for display or logging purposes. This history is not sent to the model in this implementation.

This structure ensures our chat engine can properly communicate with the language model while optionally maintaining a record of the conversation.

---

## Understanding Prompt Templates in LangChain

Let's take a closer look at how we define the prompt template in our `ChatEngine` class. This part is crucial for controlling how information is passed to the language model.

```python
self.prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(self.system_message),
    HumanMessagePromptTemplate.from_template(
        "Context:\n{context}\n\nQuestion: {question}"
    )
])
```

### Here’s what each component does and what it returns:

* **SystemMessagePromptTemplate.from\_template**
  This method takes a string template (our system instructions) and returns a `SystemMessagePromptTemplate` object. This object knows how to generate a system message for the chat model by filling in any placeholders if needed.

* **HumanMessagePromptTemplate.from\_template**
  This method takes a string template for the user's message (with placeholders for context and question) and returns a `HumanMessagePromptTemplate` object. This object can generate a human message for the chat model by filling in those placeholders.

* **ChatPromptTemplate.from\_messages**
  This method takes a list of message prompt templates (like the two above) and returns a `ChatPromptTemplate` object. This object can generate a full list of formatted messages (system and human) ready to be sent to the chat model, using the values you provide for the placeholders.

### Key Difference:

* The `.from_template` methods create individual message templates (for either system or human messages).
* The `.from_messages` method combines multiple message templates into a single prompt template that can generate the full message sequence for the chat model.

By using these together, we can clearly separate the instructions (system message) from the user input (human message), and then combine them into the exact format the language model expects.

---

## Building the Message Handling System

Now that we have our basic class structure, let's implement the core functionality: sending messages and receiving responses. The `send_message` method will handle this process.

```python
def send_message(self, user_message, context=""):
    """Send a message to the chat engine and get a response"""
    # Format the messages using the prompt template (includes system message)
    messages = self.prompt.format_messages(
        context=context,
        question=user_message
    )
    # Get the response from the model
    response = self.chat_model.invoke(messages)
    
    # Optionally, track the conversation for display/logging
    self.conversation_history.append(HumanMessage(content=user_message))
    self.conversation_history.append(AIMessage(content=response.content))
    
    # Return the AI's response content
    return response.content
```

### Explanation:

* **Format Messages**: We use our prompt template to fill in placeholders with the provided context and question. The system message is always included.
* **Get Response**: We invoke the chat model with our formatted messages using `self.chat_model.invoke(messages)`.
* **Update History**: We append the user's message and the AI's response to the conversation history for display or logging.
* **Return Result**: We return the content of the response to be displayed to the user.

Note: In this implementation, conversation history is not sent to the model. Each response is based only on the current context and question, which is typical for RAG systems.

---

## Implementing Conversation Management

An important aspect of any chat system is the ability to manage the conversation state. Let's implement a method to reset the conversation history:

```python
def reset_conversation(self):
    """Reset the conversation history (for display/logging only)"""
    self.conversation_history = []
```

### Purpose:

* **Reset**: Clears the conversation history. This is useful for display or logging purposes, and allows users to start fresh when needed.

---

## Testing Our Chat Engine Without Context

Let's see how our chat engine behaves when we don't provide any context. This is important because, in a RAG system, the assistant should not "hallucinate" answers—it should only respond based on the information it has.

### Test Code:

```python
from chat_engine import ChatEngine

# Initialize the chat engine
chat_engine = ChatEngine()

# Send a message without context (should politely decline)
query = "What is the capital of France?"
response = chat_engine.send_message(query)
print(f"Question: {query}")
print(f"Answer: {response}")

# Print conversation history
print("\nConversation history:")
print(chat_engine.conversation_history)
```

### Expected Output:

```
Question: What is the capital of France?
Answer: I'm sorry, but based on the provided context, I don't have the necessary information to answer your question accurately. If you could provide additional context, I'd be happy to help.

Conversation history:
[HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm sorry, but based on the provided context, I don't have the necessary information to answer your question accurately. If you could provide additional context, I'd be happy to help.", additional_kwargs={}, response_metadata={})]
```

---

## Testing With Context

Now, let's test the chat engine with some relevant context. This simulates the scenario where our document processor has retrieved useful information, and we want the assistant to answer using only that context.

### Test Code:

```python
# Send a message with context (should answer using only the context)
context = """Paris is the capital and most populous city of France. 
The Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral are among its most famous landmarks."""
query = "Tell me about the landmarks mentioned."

# Get response with context provided
response = chat_engine.send_message(query, context)

# Display the question and answer
print(f"\nQuestion with context: {query}")
print(f"Answer: {response}")

# Print updated conversation history
print("\nUpdated conversation history:")
print(chat_engine.conversation_history)
```

### Expected Output:

```
Question with context: Tell me about the landmarks mentioned.
Answer: The Eiffel Tower is an iconic iron structure in Paris, known for its intricate lattice metalwork and panoramic views from the top. The Louvre Museum is a historic monument housing a vast collection of art, including the famous painting of the Mona Lisa. Notre-Dame Cathedral is a stunning Gothic cathedral known for its architecture and historical significance as a religious and cultural symbol in Paris.

Updated conversation history:
[HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm sorry, but based on the provided context, I don't have the necessary information to answer your question accurately. If you could provide additional context, I'd be happy to help.", additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me about the landmarks mentioned.', additional_kwargs={}, response_metadata={}), AIMessage(content='The Eiffel Tower is an iconic iron structure in Paris, known for its intricate lattice metalwork and panoramic views from the top. The Louvre Museum is a historic monument housing a vast collection of art, including the famous painting of the Mona Lisa. Notre-Dame Cathedral is a stunning Gothic cathedral known for its architecture and historical significance as a religious and cultural symbol in Paris.', additional_kwargs={}, response_metadata={})]
```

---

## Resetting the Conversation

Finally, let's see how to reset the conversation history. This is useful if you want to clear the previous exchanges.

```python
# Reset the conversation history (for display/logging only)
chat_engine.reset_conversation()
print("\nConversation history has been reset.")

# Print conversation history after reset
print("\nConversation history after reset:")
print(chat_engine.conversation_history)
```

### Expected Output:

```
Conversation history has been reset.

Conversation history after reset:
[]
```

---

## Summary and Practice Preview

In this lesson, we've built a chat engine for our RAG chatbot using explicit system and human message templates. We've learned how to:

* Create a `ChatEngine` class that manages conversations with a language model
* Define system messages to guide the AI's behavior
* Format prompts with context and questions using templates
* Optionally maintain conversation history for display or logging
* Implement methods to send messages and reset conversation history
* Test our chat engine with various scenarios

Our chat engine complements the document processor we built in the previous lesson. While the document processor handles the retrieval of relevant information, the chat engine manages the conversation and presents this information to the user in a natural way. In the next unit, we'll integrate the document processor and chat engine to create a complete RAG system. This integration will allow our chatbot to automatically retrieve relevant context from documents based on user queries, creating a seamless experience where users can ask questions about their documents and receive informed, contextual responses.

Get ready to practice what you've learned and take your RAG chatbot to the next level!


## Initializing the Chat Engine

You've done a fantastic job understanding the ChatEngine class! Now, let's focus on completing the **init** method. Your task is to:

Instantiate the chat model using ChatOpenAI.
Define a system message that instructs the AI to answer questions only based on the provided context.
Initialize the conversation history as an empty list (for display/logging only).
Once you've completed these steps, verify your work by printing both the system message and the conversation history to ensure they are set up correctly. This will confirm that your initialization is working as intended. Dive in and see how your chat engine starts to take shape!

```python
from langchain_openai import ChatOpenAI
from langchain.schema.messages import SystemMessage, HumanMessage, AIMessage
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate


class ChatEngine:
    def __init__(self):
        # TODO: Instantiate the chat model using ChatOpenAI
        # TODO: Define the system message with strict instructions
        # TODO: Initialize conversation history as an empty list


from chat_engine import ChatEngine

# Initialize the chat engine
chat_engine = ChatEngine()

# TODO: Print the system message to verify it is set correctly

# TODO: Print the conversation history to verify it starts as an empty list


```

Here's how you can complete the `__init__` method for the `ChatEngine` class. This will involve initializing the chat model, defining the system message, and setting up the conversation history as an empty list.

### Updated Code for `ChatEngine`:

```python
from langchain_openai import ChatOpenAI
from langchain.schema.messages import SystemMessage, HumanMessage, AIMessage
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate


class ChatEngine:
    def __init__(self):
        # Instantiate the chat model using ChatOpenAI
        self.chat_model = ChatOpenAI()

        # Define the system message with strict instructions
        self.system_message = (
            "You are a helpful assistant that ONLY answers questions based on the "
            "provided context. If no relevant context is provided, do NOT answer the "
            "question and politely inform the user that you don't have the necessary "
            "information to answer their question accurately."
        )
        
        # Initialize conversation history as an empty list (for display/logging only)
        self.conversation_history = []

        # Optionally, define the prompt template (can be expanded as needed)
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(self.system_message),
            HumanMessagePromptTemplate.from_template(
                "Context:\n{context}\n\nQuestion: {question}"
            )
        ])


# Initialize the chat engine in main.py
chat_engine = ChatEngine()

# Print the system message to verify it is set correctly
print("System Message:", chat_engine.system_message)

# Print the conversation history to verify it starts as an empty list
print("\nConversation History:", chat_engine.conversation_history)
```

### Explanation of the Code:

1. **Chat Model**:
   The `self.chat_model = ChatOpenAI()` instantiates the OpenAI chat model, which will be used to generate responses based on the prompts we define.

2. **System Message**:
   The `self.system_message` defines the behavior of the AI model. It instructs the assistant to answer questions only based on the provided context and politely inform the user if there is no relevant context.

3. **Conversation History**:
   `self.conversation_history = []` initializes an empty list to track the conversation for display or logging purposes. This list will store both the user's messages and the AI's responses.

4. **Prompt Template**:
   The `self.prompt` defines how the conversation messages are structured (system message and human message). This is a basic setup that ensures the assistant works according to the instructions.

### Verifying the Setup:

Once you run the code, you should see the following outputs printed to verify that everything is set up correctly:

```
System Message: You are a helpful assistant that ONLY answers questions based on the provided context. If no relevant context is provided, do NOT answer the question and politely inform the user that you don't have the necessary information to answer their question accurately.

Conversation History: []
```

This confirms that the `ChatEngine` class is properly initialized with the correct system message and an empty conversation history, as intended.


## Integrating Prompt Templates

Now, let's enhance the __init__ method by integrating a prompt template using explicit system and human message templates. Here's what you need to do:

In chat_engine.py:

Use SystemMessagePromptTemplate and HumanMessagePromptTemplate together with ChatPromptTemplate.from_messages to create a prompt template that:
Includes a system message instructing the model to answer questions based only on the provided context.
Includes a human message template with placeholders for both context and question parameters.
Store this template in the self.prompt attribute.
In main.py:

Format the prompt template with sample context and question values using self.prompt.format_messages.
Print the formatted messages as a single object to verify their structure.
By completing these steps, you'll ensure your chat engine is ready to handle context-based queries effectively using explicit message templates.

```python
from langchain_openai import ChatOpenAI
from langchain.schema.messages import SystemMessage, HumanMessage, AIMessage
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate


class ChatEngine:
    def __init__(self):
        self.chat_model = ChatOpenAI()
        self.system_message = (
            "You are a helpful assistant that ONLY answers questions based on the "
            "provided context. If no relevant context is provided, do NOT answer the "
            "question and politely inform the user that you don't have the necessary "
            "information to answer their question accurately."
        )
        
        # TODO: Define the prompt template using ChatPromptTemplate.from_messages,
        # SystemMessagePromptTemplate, and HumanMessagePromptTemplate.
        # The template should include a system message and a human message with placeholders for context and question.
        
        # Optionally, keep conversation history for display/logging only
        self.conversation_history = []


from chat_engine import ChatEngine

# Initialize the chat engine
chat_engine = ChatEngine()

# Define a sample context
sample_context = "The board game 'Catan' involves players collecting resources to build roads and settlements."

# Define a sample question related to the context
sample_question = "What do players do in the game 'Catan'?"

# TODO: Format the prompt template with the sample context and question

# TODO: Print the formatted messages to verify their structure

```

To integrate the prompt templates in the `ChatEngine` class, we'll follow the steps mentioned: creating a `SystemMessagePromptTemplate` and a `HumanMessagePromptTemplate` and then using `ChatPromptTemplate.from_messages` to assemble them into a complete prompt template. We'll also format this template with the given context and question in `main.py`.

Here’s how to implement the required changes:

### Updated `chat_engine.py`

```python
from langchain_openai import ChatOpenAI
from langchain.schema.messages import SystemMessage, HumanMessage, AIMessage
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

class ChatEngine:
    def __init__(self):
        # Instantiate the chat model using ChatOpenAI
        self.chat_model = ChatOpenAI()

        # Define the system message with strict instructions
        self.system_message = (
            "You are a helpful assistant that ONLY answers questions based on the "
            "provided context. If no relevant context is provided, do NOT answer the "
            "question and politely inform the user that you don't have the necessary "
            "information to answer their question accurately."
        )
        
        # Define the prompt template using SystemMessagePromptTemplate and HumanMessagePromptTemplate
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(self.system_message),
            HumanMessagePromptTemplate.from_template(
                "Context:\n{context}\n\nQuestion: {question}"
            )
        ])
        
        # Optionally, keep conversation history for display/logging only
        self.conversation_history = []
```

### Explanation:

* **System Message**: The system message instructs the assistant to only answer based on the provided context.
* **Prompt Template**: The `ChatPromptTemplate` is created using `SystemMessagePromptTemplate` and `HumanMessagePromptTemplate`. The `HumanMessage` template includes placeholders for `context` and `question`.
* **Conversation History**: This is an optional feature to track the history of the conversation for display/logging.

---

### Updated `main.py`

In `main.py`, we will format the prompt template with sample context and question values and then print the formatted messages.

```python
from chat_engine import ChatEngine

# Initialize the chat engine
chat_engine = ChatEngine()

# Define a sample context
sample_context = "The board game 'Catan' involves players collecting resources to build roads and settlements."

# Define a sample question related to the context
sample_question = "What do players do in the game 'Catan'?"

# Format the prompt template with the sample context and question
formatted_messages = chat_engine.prompt.format_messages(
    context=sample_context,
    question=sample_question
)

# Print the formatted messages to verify their structure
for message in formatted_messages:
    print(f"{message.__class__.__name__}: {message.content}")
```

### Explanation:

* **Sample Context and Question**: We define `sample_context` and `sample_question` to simulate a user query and relevant context.
* **Formatting the Prompt**: The `self.prompt.format_messages` method fills in the placeholders (`context` and `question`) in the prompt template.
* **Printing the Messages**: We loop through the `formatted_messages` and print each message’s class name and content to ensure the formatting is correct.

---

### Expected Output

When you run `main.py`, you should see output similar to this:

```
SystemMessage: You are a helpful assistant that ONLY answers questions based on the provided context. If no relevant context is provided, do NOT answer the question and politely inform the user that you don't have the necessary information to answer their question accurately.
HumanMessage: Context:
The board game 'Catan' involves players collecting resources to build roads and settlements.

Question: What do players do in the game 'Catan'?
```

This output confirms that the system message and human message are properly formatted and ready to be used in the chat engine. You have successfully integrated the prompt templates into the `ChatEngine` class!


## Implementing the Send Message Method

Next, let's implement the message handling method, which is crucial for interactions with the chat model.

Here's what you need to do:

Implement a method that accepts two parameters:
A string containing the user's query
An optional string parameter for the context relevant to answering the query
Use the prompt template (which includes both a system and a human message) to format messages, incorporating both the context and the user's query
Use the chat model to get a response based on the formatted messages
Append the user's message and the AI's response to the conversation history
Return the content of the response
After implementing this method, test it by adding code in main.py to call it with a sample context and question. This will help you verify that your implementation correctly formats the message, interacts with the chat model, and returns the expected response. Let's see how your chat engine comes to life!


```python
from langchain_openai import ChatOpenAI
from langchain.schema.messages import SystemMessage, HumanMessage, AIMessage
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate


class ChatEngine:
    def __init__(self):
        self.chat_model = ChatOpenAI()
        self.system_message = (
            "You are a helpful assistant that ONLY answers questions based on the "
            "provided context. If no relevant context is provided, do NOT answer the "
            "question and politely inform the user that you don't have the necessary "
            "information to answer their question accurately."
        )
        
        # Define the prompt template with explicit system and human messages
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(self.system_message),
            HumanMessagePromptTemplate.from_template(
                "Context:\n{context}\n\nQuestion: {question}"
            )
        ])
        
        # Optionally, keep conversation history for display/logging only
        self.conversation_history = []

    # TODO: Implement a method to send a message to the chat engine and get a response
    # This method should take a user_message (str) and optional context (str)
        # TODO: Format the message using the prompt template
        
        # TODO: Get the response from the model
        
        # TODO: Add both the user message and AI response to the conversation history
        
        # TODO: Return the response content



from chat_engine import ChatEngine

# Initialize the chat engine
chat_engine = ChatEngine()

# Define a sample context
context = ("Mars is the fourth planet from the Sun and the second-smallest planet "
           "in the Solar System. It is often called the 'Red Planet' because of "
           "its reddish appearance due to iron oxide (rust) on its surface. "
           "Mars has two small moons named Phobos and Deimos.")

# Define a sample question related to the context
query = "How is Mars often called?"

# TODO: Send a message with context and get a response

# TODO: Print the question and answer        

```

To implement the `send_message` method in the `ChatEngine` class, we'll follow the steps outlined:

1. **Format the message**: We'll format the message using the prompt template, which includes both the system message and the user's query.
2. **Get the response**: We'll use the `chat_model` to get a response based on the formatted message.
3. **Update the conversation history**: We'll append the user's message and the AI's response to the conversation history.
4. **Return the response content**: Finally, we will return the content of the response from the AI model.

Here's the updated code for the `ChatEngine` class and the main script to test the functionality:

### Updated `chat_engine.py`

```python
from langchain_openai import ChatOpenAI
from langchain.schema.messages import SystemMessage, HumanMessage, AIMessage
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

class ChatEngine:
    def __init__(self):
        # Instantiate the chat model using ChatOpenAI
        self.chat_model = ChatOpenAI()

        # Define the system message with strict instructions
        self.system_message = (
            "You are a helpful assistant that ONLY answers questions based on the "
            "provided context. If no relevant context is provided, do NOT answer the "
            "question and politely inform the user that you don't have the necessary "
            "information to answer their question accurately."
        )
        
        # Define the prompt template using SystemMessagePromptTemplate and HumanMessagePromptTemplate
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(self.system_message),
            HumanMessagePromptTemplate.from_template(
                "Context:\n{context}\n\nQuestion: {question}"
            )
        ])
        
        # Optionally, keep conversation history for display/logging only
        self.conversation_history = []

    def send_message(self, user_message, context=""):
        """Send a message to the chat engine and get a response"""
        # Format the messages using the prompt template (includes system message)
        messages = self.prompt.format_messages(
            context=context,
            question=user_message
        )
        
        # Get the response from the model
        response = self.chat_model.invoke(messages)
        
        # Add both the user message and AI response to the conversation history
        self.conversation_history.append(HumanMessage(content=user_message))
        self.conversation_history.append(AIMessage(content=response.content))
        
        # Return the response content
        return response.content
```

### Testing the `send_message` Method in `main.py`

Now, let's add the code in `main.py` to call the `send_message` method with a sample context and query:

```python
from chat_engine import ChatEngine

# Initialize the chat engine
chat_engine = ChatEngine()

# Define a sample context
context = ("Mars is the fourth planet from the Sun and the second-smallest planet "
           "in the Solar System. It is often called the 'Red Planet' because of "
           "its reddish appearance due to iron oxide (rust) on its surface. "
           "Mars has two small moons named Phobos and Deimos.")

# Define a sample question related to the context
query = "How is Mars often called?"

# Send a message with context and get a response
response = chat_engine.send_message(query, context)

# Print the question and answer        
print(f"Question: {query}")
print(f"Answer: {response}")

# Optionally, print the conversation history for verification
print("\nConversation History:")
for message in chat_engine.conversation_history:
    print(f"{message.__class__.__name__}: {message.content}")
```

### Explanation:

1. **`send_message` Method**:

   * The method formats the message using the `prompt` template.
   * It sends the formatted message to the chat model, retrieves the response, and appends both the user's message and the AI's response to the conversation history.
   * It returns the AI's response content.

2. **Testing in `main.py`**:

   * We define a sample context and query related to Mars.
   * The `send_message` method is called with the query and context, and the response is printed.
   * The conversation history is printed to verify the messages.

### Expected Output:

When you run the code, the output should look something like this:

```
Question: How is Mars often called?
Answer: Mars is often called the 'Red Planet' due to its reddish appearance caused by iron oxide (rust) on its surface.

Conversation History:
HumanMessage: How is Mars often called?
AIMessage: Mars is often called the 'Red Planet' due to its reddish appearance caused by iron oxide (rust) on its surface.
```

This confirms that the `send_message` method works correctly, interacting with the chat model, formatting messages, and maintaining the conversation history.



## Testing Chat Engine Without Context

This time, let's put the send_message method to the test by calling it without any context.

Modify the existing code to call the send_message method with your query but without passing any context (remove the context parameter from the function call).

This exercise will help you see how the chat engine behaves when context is missing, highlighting the importance of context in generating accurate responses. Dive in and see how your chat engine handles this scenario!

```python
from chat_engine import ChatEngine

# Initialize the chat engine
chat_engine = ChatEngine()

# Define a sample context
context = ("Mars is the fourth planet from the Sun and the second-smallest planet "
           "in the Solar System. It is often called the 'Red Planet' because of "
           "its reddish appearance due to iron oxide (rust) on its surface. "
           "Mars has two small moons named Phobos and Deimos.")

# Define a sample question related to the context
query = "How is Mars often called?"

# TODO: Send a message without the context and get a response
response = chat_engine.send_message(query, context)

# Print the question and answer
print(f"Question with context: {query}")
print(f"Answer: {response}")


from langchain_openai import ChatOpenAI
from langchain.schema.messages import SystemMessage, HumanMessage, AIMessage
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate


class ChatEngine:
    def __init__(self):
        self.chat_model = ChatOpenAI()
        self.system_message = (
            "You are a helpful assistant that ONLY answers questions based on the "
            "provided context. If no relevant context is provided, do NOT answer the "
            "question and politely inform the user that you don't have the necessary "
            "information to answer their question accurately."
        )
        
        # Define the prompt template with explicit system and human messages
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(self.system_message),
            HumanMessagePromptTemplate.from_template(
                "Context:\n{context}\n\nQuestion: {question}"
            )
        ])
        
        # Optionally, keep conversation history for display/logging only
        self.conversation_history = []

    def send_message(self, user_message, context=""):
        """Send a message to the chat engine and get a response"""
        # Format the messages using the prompt template (includes system message)
        messages = self.prompt.format_messages(
            context=context,
            question=user_message
        )
        # Get the response from the model
        response = self.chat_model.invoke(messages)
        
        # Track the conversation for display/logging
        self.conversation_history.append(HumanMessage(content=user_message))
        self.conversation_history.append(AIMessage(content=response.content))
        return response.content



```

Here’s how to modify the test code in `main.py` to call `send_message` **without any context**, so you can see how the chat engine behaves when no relevant information is provided:

```python
from chat_engine import ChatEngine

# Initialize the chat engine
chat_engine = ChatEngine()

# Define a sample question (no context provided)
query = "How is Mars often called?"

# Send a message without passing context (default is empty string)
response = chat_engine.send_message(query)  # context parameter omitted

# Print the question and answer
print(f"Question without context: {query}")
print(f"Answer: {response}")

# Optionally, print conversation history to verify
print("\nConversation History:")
for message in chat_engine.conversation_history:
    print(f"{message.__class__.__name__}: {message.content}")
```

---

### What to expect:

Because the system message instructs the AI to only answer based on provided context, and here no context is given, the assistant should politely decline to answer, something like:

```
Question without context: How is Mars often called?
Answer: I'm sorry, but based on the provided context, I don't have the necessary information to answer your question accurately. If you could provide additional context, I'd be happy to help.

Conversation History:
HumanMessage: How is Mars often called?
AIMessage: I'm sorry, but based on the provided context, I don't have the necessary information to answer your question accurately. If you could provide additional context, I'd be happy to help.
```

---

### Summary

* Remove the `context` argument when calling `send_message`.
* The assistant respects the system instructions and will not guess or hallucinate without context.
* Conversation history will still track the question and the polite refusal answer.

This test demonstrates the importance of providing relevant context in a RAG-powered chatbot to get accurate and useful responses.


## Resetting Conversation History

Let's add an important feature to your chat engine—the ability to clear the conversation history!

In the ChatEngine class, implement the method that resets the conversation history by clearing the conversation_history list.
Test your implementation by calling this method, and then printing the conversation history to verify it is empty.
This is a small but powerful addition that will make your chatbot more flexible and prevent it from getting confused by lengthy conversation histories. Give it a try!


```python
from langchain_openai import ChatOpenAI
from langchain.schema.messages import SystemMessage, HumanMessage, AIMessage
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate


class ChatEngine:
    def __init__(self):
        self.chat_model = ChatOpenAI()
        self.system_message = (
            "You are a helpful assistant that ONLY answers questions based on the "
            "provided context. If no relevant context is provided, do NOT answer the "
            "question and politely inform the user that you don't have the necessary "
            "information to answer their question accurately."
        )
        
        # Define the prompt template with explicit system and human messages
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(self.system_message),
            HumanMessagePromptTemplate.from_template(
                "Context:\n{context}\n\nQuestion: {question}"
            )
        ])
        
        # Optionally, keep conversation history for display/logging only
        self.conversation_history = []

    def send_message(self, user_message, context=""):
        """Send a message to the chat engine and get a response"""
        # Format the messages using the prompt template (includes system message)
        messages = self.prompt.format_messages(
            context=context,
            question=user_message
        )
        # Get the response from the model
        response = self.chat_model.invoke(messages)
        
        # Optionally, track the conversation for display/logging
        self.conversation_history.append(HumanMessage(content=user_message))
        self.conversation_history.append(AIMessage(content=response.content))
        return response.content

    # TODO: Implement a method to reset the conversation history



from chat_engine import ChatEngine

# Initialize the chat engine
chat_engine = ChatEngine()

# Define a sample context about Mars
context = ("Mars is the fourth planet from the Sun and the second-smallest planet "
           "in the Solar System. It is often called the 'Red Planet' because of "
           "its reddish appearance due to iron oxide (rust) on its surface. "
           "Mars has two small moons named Phobos and Deimos.")

# Define a sample question related to the context
query = "Why does the planet have that color?"

# Send a message with context and get a response
response = chat_engine.send_message(query, context)

# Print the question and answer
print(f"Question with context: {query}")
print(f"Answer: {response}")

# Print the conversation history
print("\nConversation history:")
print(chat_engine.conversation_history)

# Send another question about Mars
query2 = "What are the names of Mars' moons?"
response2 = chat_engine.send_message(query2, context)
print(f"\nQuestion with context: {query2}")
print(f"Answer: {response2}")

# Print updated conversation history
print("\nUpdated conversation history:")
print(chat_engine.conversation_history)

# TODO: Reset the conversation history

# TODO: Print conversation history after reset



```

```python
from langchain_openai import ChatOpenAI
from langchain.schema.messages import SystemMessage, HumanMessage, AIMessage
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate


class ChatEngine:
    def __init__(self):
        self.chat_model = ChatOpenAI()
        self.system_message = (
            "You are a helpful assistant that ONLY answers questions based on the "
            "provided context. If no relevant context is provided, do NOT answer the "
            "question and politely inform the user that you don't have the necessary "
            "information to answer their question accurately."
        )
        
        # Define the prompt template with explicit system and human messages
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(self.system_message),
            HumanMessagePromptTemplate.from_template(
                "Context:\n{context}\n\nQuestion: {question}"
            )
        ])
        
        # Optionally, keep conversation history for display/logging only
        self.conversation_history = []

    def send_message(self, user_message, context=""):
        """Send a message to the chat engine and get a response"""
        # Format the messages using the prompt template (includes system message)
        messages = self.prompt.format_messages(
            context=context,
            question=user_message
        )
        # Get the response from the model
        response = self.chat_model.invoke(messages)
        
        # Optionally, track the conversation for display/logging
        self.conversation_history.append(HumanMessage(content=user_message))
        self.conversation_history.append(AIMessage(content=response.content))
        return response.content

    def reset_conversation(self):
        """Reset the conversation history (for display/logging only)"""
        self.conversation_history = []


# ----------------------- Testing in main.py -----------------------

from chat_engine import ChatEngine

# Initialize the chat engine
chat_engine = ChatEngine()

# Define a sample context about Mars
context = ("Mars is the fourth planet from the Sun and the second-smallest planet "
           "in the Solar System. It is often called the 'Red Planet' because of "
           "its reddish appearance due to iron oxide (rust) on its surface. "
           "Mars has two small moons named Phobos and Deimos.")

# Define a sample question related to the context
query = "Why does the planet have that color?"

# Send a message with context and get a response
response = chat_engine.send_message(query, context)

# Print the question and answer
print(f"Question with context: {query}")
print(f"Answer: {response}")

# Print the conversation history
print("\nConversation history:")
print(chat_engine.conversation_history)

# Send another question about Mars
query2 = "What are the names of Mars' moons?"
response2 = chat_engine.send_message(query2, context)
print(f"\nQuestion with context: {query2}")
print(f"Answer: {response2}")

# Print updated conversation history
print("\nUpdated conversation history:")
print(chat_engine.conversation_history)

# Reset the conversation history
chat_engine.reset_conversation()

# Print conversation history after reset
print("\nConversation history after reset:")
print(chat_engine.conversation_history)
```

---

### Explanation:

* The `reset_conversation` method simply clears the `conversation_history` list.
* After sending two messages and printing the history, we call `reset_conversation()` to clear it.
* Printing the history after reset should show an empty list `[]`.

---

### Expected Output Snippet:

```
Question with context: Why does the planet have that color?
Answer: Mars appears red due to iron oxide (rust) on its surface, giving it its distinctive color.

Conversation history:
[HumanMessage(content='Why does the planet have that color?'), AIMessage(content='Mars appears red due to iron oxide (rust) on its surface, giving it its distinctive color.')]

Question with context: What are the names of Mars' moons?
Answer: Mars has two small moons named Phobos and Deimos.

Updated conversation history:
[HumanMessage(content="What are the names of Mars' moons?"), AIMessage(content="Mars has two small moons named Phobos and Deimos.")]

Conversation history after reset:
[]
```

This confirms your chat engine now supports resetting conversation history!
